# Complexité en temps

## Génération des instances

### Mode $I_m$
```python
    print("Saisissez m: ")
    m = int(input())
    tasks = np.concatenate((
        [m for i in range(3)], # 3 itérations
        np.repeat([m + i for i in range(1, m)], 2) # 2*m itération
    ))

    print(tasks)
    instance = np.concatenate(([m, len(tasks)], tasks))
```

Donc la génération de l'instance $I_m$ prend $\mathcal{O}(3+2m)$ temps.

### Mode $I_R$
```python
    instances = []
    print("Saisissez n, m, k, min, max. Par exemple: 'n m k min max'")
    n, m, k, low, high = [int(item) for item in input().split()]
    print(n, m, k, low, high)
    for i in range(k):
        instances.append( # append prend O(1) temps
            # Génération un array de n élémént prend O(n) temps
            np.concatenate(([m, n], np.random.random_integers(low, high, size=n)))
        )
```

Donc la génération de l'instance $I_m$ prend $\mathcal{O}(kn)$ temps.

## Copie en mémoire

La fonction suivante permet de transformer une chaîne de caractère (fournie par $I_f$ ou $I_c$:

```python
def getTokenIfValid(seq: str):
    """Parse l'entrée et transforme en array"""
    tokens = seq.split(":")
    m, n, tasks = int(tokens[0]), int(tokens[1]), tokens[2:len(tokens)]
    if(n != len(tasks)):
        print("Il n'y a pas assez de tâches, il en faut n(", n, ")")
        return None

    return tokens   
```

La complexité dépend de la fonction ```split``` de Python, donc le [code source](https://github.com/python/cpython/blob/master/Objects/stringlib/split.h) est comme suit:

```c
    while (maxcount-- > 0) {
        while (i < str_len && STRINGLIB_ISSPACE(str[i]))
            i++;
        if (i == str_len) break;
        j = i; i++;
        while (i < str_len && !STRINGLIB_ISSPACE(str[i]))
            i++;
```

On peut déduire facilement que ```split``` s'éxécute en $\mathcal{O}(s)$ temps, avec $s$ étant la longueur de la chaîne de caractères en question. Finalemet,
La copie en mémoire des instances prend $\mathcal{O}(s)$ temps.

## Les algorithmes pour le problème Min-Makespan

### ```LSA```

```python
def LSA(m: int, tasks: list):
    """Les tâches seront traités dans l'ordre tel qu'elles sont fournies"""

    M = dict([(i, []) for i in range(1, m+1)])
    
    for task in tasks: # n itération
        availIdx = 1
        loads = dict(map(lambda item: (item[0], sum(item[1])), M.items())) 
        availIdx = min(loads, key=loads.get)
        M[availIdx].append(task)
    
    return M
```

Le mapping consiste à parcourir chacun de $m$ machines puis faire une somme des durées. 
- $d'_1$: ```loads``` contient 1 tâche,
- $d'_2$: ```loads``` contient 2 tâches 
- ...
- $d'_n$: ```loads``` contient $n$ tâches

A chaque itération on fait ```min``` et ```sum``` du nombre de tâches au courant. Le nombre d'itérations au total est donc 2 fois (2 opérations) la somme d'une suite arithmétique $(1, 2, ..., n)$, donc $\frac{2n(n+1)}{2} = n(n+1) = n^2 + n$. Au final, ```LSA``` prend $\mathcal{O}(n^2)$ temps.


### ```LPT```

```python
def LPT(m: int, tasks: list):
    """Les tâches seront traités dans l'ordre décroissant de leur durée"""

    M = dict([(i, []) for i in range(1, m+1)])
    # Trier selon l'ordre décroissant, O(n*log(n)) mergesort 
    tasks = np.sort(tasks, kind="mergesort")[::-1] 
    #tasks = np.sort(tasks)[::-1] # O(n^2) quicksort par défaut
    
    for task in tasks:
        availIdx = 1
        loads = dict(map(lambda item: (item[0], sum(item[1])), M.items())) 
        availIdx = min(loads, key=loads.get)
        M[availIdx].append(task)
    
    return M
```

```LPT``` est juste une variation de ```LST```, en ordonnant les tâches dans l'ordre décroissant, qui prend $\mathcal{O}(n^2)$ ou $\mathcal{O}(n log(n))$ selon le choix de [l'algorithme de tri](https://docs.scipy.org/doc/numpy/reference/generated/numpy.sort.html#Notes). Au final, ```LPT``` prend $\mathcal{O}(n^2)$ temps.


### ```MyAlgo```

```python
def MyAlgo(m: int, tasks: list):
    """Affecter les tâches en utilisant le principe du Min Binpacking. 
    On va essayer de faire m bins de capacité ~opt """

    M = dict([(i, []) for i in range(1, m+1)])
    opt = np.ceil(len(tasks)/m)
    tasks = tasks = np.sort(tasks, kind="mergesort")
    
    # 
    macIdx = 1
    for task in tasks: # n itérations
        if(macIdx < m - 1):
            while(sum(M[macIdx]) <= opt):
                M[macIdx].append(task) # O(1)
            macIdx += 1
        else:
            M[macIdx].append(task)
    
    return M
```

Le mapping consiste à parcourir chacun de $m$ machines puis faire une somme des durées. 
- $d'_1$: ```loads``` contient 1 tâche,
- $d'_2$: ```loads``` contient 2 tâches 
- ...
- $d'_n$: ```loads``` contient $n$ tâches

A chaque itération on fait ```sum``` du nombre de tâches au courant. Le nombre d'itérations au total est donc la somme d'une suite arithmétique $(1, 2, ..., n)$, donc $\frac{n(n+1)}{2} = \frac{n^2 + n}{2}$. Au final, ```LSA``` prend $\mathcal{O}(n^2)$ temps.


# Résultats